In [23]:
import numpy as np
import matplotlib.pyplot as plt
from GraphDomain import EncodePriority
import os

### **MFEA**

In [26]:
def Load_data(sc = 'NULL'):
    sc = "Datasets/IDPCDU_Edges/set1/"
    Tasks = os.listdir(sc)
    MAX_DIM = 0
    NUM_TASK = len(Tasks)

    for i in range(len(Tasks)):
        path = sc + Tasks[i] 
        name = Tasks[i].split('.')[0]
        Tasks[i] = EncodePriority(path, name=name)
        MAX_DIM = max(MAX_DIM, Tasks[i].NUM_DOMAIN)

    return Tasks, NUM_TASK, MAX_DIM


In [27]:
Tasks, NUM_TASK, MAX_DIM = Load_data()

In [28]:
def GeneratorIndiv(dims=MAX_DIM):
    indiv = np.random.permutation(range(1,dims+1))

    fac_cost = []
    for task in Tasks:
        fac_cost.append(task.Cost(indiv))
    
    return indiv, fac_cost

In [29]:
def Generate_Eval(sizePop, d = MAX_DIM):
    population = []
    fac_cost = []

    for i in range(sizePop):
        pi, fcosti = GeneratorIndiv()
        population.append(pi)
        fac_cost.append(fcosti)
    
    population, fac_cost = np.array(population), np.array(fac_cost)
    rank = np.argsort(fac_cost,axis = 0) + 1
    skill_factor = np.argmin(rank,axis = 1) + 1
    fitness = 1/np.min(rank, axis = 1)

    return population, fac_cost, skill_factor, fitness
    

In [30]:
def chooseParents(fitness):
    n = len(fitness)
    index1 = np.random.randint(n)
    index2 = np.random.randint(n)
    while index1 == index2:
        index2 = np.random.randint(n)
    
    index3 = np.random.randint(n)
    index4 = np.random.randint(n)
    while index3 == index4:
        index4 = np.random.randint(n)
    
    if fitness[index1] > fitness[index2]:
        p1_id = index1
    else:
        p1_id = index2
    
    if fitness[index3] > fitness[index4]:
        p2_id = index3
    else:
        p2_id = index4
    
    return p1_id, p2_id

In [31]:
def cross_pmx(p1, p2):
    n = len(p1)
    
    index1 = np.random.randint(1, n-1)
    index2 = np.random.randint(1, n-1)
    while index1 == index2:
       index2 = np.random.randint(1, n-1)
   
    if index1 > index2:
        temp = index1
        index1 = index2
        index2 = temp

    #print(index1, index2)

    o1 = np.array(p2)
    o2 = np.array(p1)

    pos1 = np.full(n+1, -1, dtype=int)
    pos2 = np.full(n+1, -1, dtype=int)
    for i in range(index1, index2+1):
        pos1[o1[i]] = i
        pos2[o2[i]] = i
        
    for i in range(n):
        if index1 <= i and i <= index2: continue

        id1 = i
        while True:
            if pos1[p1[id1]] == -1:
                o1[i] = p1[id1]
                break
            id1 = pos1[p1[id1]]
        
        id2 = i
        while True:
            if pos2[p2[id2]] == -1:
                o2[i] = p2[id2]
                break
            id2 = pos2[p2[id2]]
    
    return o1, o2

In [32]:
def mutation_swap(individual):
    n = len(individual)
    res = np.array(individual)

    index1 = np.random.randint(n)
    index2 = np.random.randint(n)
    while index1 == index2:
        index2 = np.random.randint(n)

    temp = res[index1]
    res[index1] = res[index2]
    res[index2] = temp

    return res

In [33]:
def update_fitness(pop):
    task = [[], []]

    for indiv in pop:
        task[indiv[1]].append(indiv)
    
    task[0].sort(key=lambda p: p[3])
    task[1].sort(key=lambda p: p[3])

    for i in range(len(task[0])):
        task[0][i][2] = 1/(i+1)
    for i in range(len(task[1])):
        task[1][i][2] = 1/(i+1)

    return task[0] + task[1]

In [34]:
def selection(pre_pop, offs_pop):
    pop = pre_pop + offs_pop
    pop = update_fitness(pop)
    pop.sort(key=lambda indiv: indiv[2], reverse = True)
    next_pop = pop[:len(pre_pop)]

    return next_pop

In [38]:
def MFEA(size_pop, num_gens, rmp):
    population, fac_cost, skill_factor, fitness = Generate_Eval(sizePop=size_pop)
    logg = []

    t = 0
    while t < num_gens:
        offs_pop = []
        n_offs = 0

        while n_offs < size_pop:
            p1_id, p2_id = chooseParents(fitness)

            if skill_factor[p1_id] == skill_factor[p2_id]:
                c1, c2 = cross_pmx(population[p1_id], population[p2_id])

                offs_pop.append(c1)
                c1_fac_cost = np.full((NUM_TASK, ), np.inf)
                c1_fac_cost[skill_factor[p1_id]] = Tasks[skill_factor[p1_id]].Cost(c1)
                fac_cost.append(c1_fac_cost)
                skill_factor.append(skill_factor[p1_id])
                
                offs_pop.append(c2)
                c2_fac_cost = np.full((NUM_TASK, ), np.inf)
                c2_fac_cost[skill_factor[p2_id]] = Tasks[skill_factor[p2_id]].Cost(c2)
                fac_cost.append(c2_fac_cost)
                skill_factor.append(skill_factor[p2_id])
            elif np.random.random_sample() < rmp:
                c1, c2 = cross_pmx(population[p1_id], population[p2_id])

                
                if np.random.random_sample() < 0.5:
                    offs_pop.append(c1)
                    c1_fac_cost = np.full((NUM_TASK, ), np.inf)
                    c1_fac_cost[skill_factor[p1_id]] = Tasks[skill_factor[p1_id]].Cost(c1)
                    fac_cost.append(c1_fac_cost)
                    skill_factor.append(skill_factor[p1_id])
                else:
                    offs_pop.append(c1)
                    c1_fac_cost = np.full((NUM_TASK, ), np.inf)
                    c1_fac_cost[skill_factor[p2_id]] = Tasks[skill_factor[p2_id]].Cost(c1)
                    fac_cost.append(c1_fac_cost)
                    skill_factor.append(skill_factor[p2_id])
        
                if np.random.random_sample() < 0.5:
                    offs_pop.append(c2)
                    c2_fac_cost = np.full((NUM_TASK, ), np.inf)
                    c2_fac_cost[skill_factor[p1_id]] = Tasks[skill_factor[p1_id]].Cost(c2)
                    fac_cost.append(c2_fac_cost)
                    skill_factor.append(skill_factor[p1_id])
                else:
                    offs_pop.append(c2)
                    c2_fac_cost = np.full((NUM_TASK, ), np.inf)
                    c2_fac_cost[skill_factor[p2_id]] = Tasks[skill_factor[p2_id]].Cost(c2)
                    fac_cost.append(c2_fac_cost)
                    skill_factor.append(skill_factor[p2_id])
            
            else:
                c1 = mutation_swap(population[p1_id])
                offs_pop.append(c1)
                c1_fac_cost = np.full((NUM_TASK, ), np.inf)
                c1_fac_cost[skill_factor[p1_id]] = Tasks[skill_factor[p1_id]].Cost(c1)
                fac_cost.append(c1_fac_cost)
                skill_factor.append(skill_factor[p1_id])

                c2 = mutation_swap(population[p2_id])
                offs_pop.append(c2)
                c2_fac_cost = np.full((NUM_TASK, ), np.inf)
                c2_fac_cost[skill_factor[p2_id]] = Tasks[skill_factor[p2_id]].Cost(c2)
                fac_cost.append(c2_fac_cost)
                skill_factor.append(skill_factor[p2_id])
            
            n_offs += 2

        population = selection(population, offs_pop)
        best = [0, 0]
        
        if population[1][2] == 1.0:
            best[population[0][1]] = population[0][0]
            best[population[1][1]] = population[1][0]
        else:
            best[0] = population[0][0]
            best[1] = population[0][0]
        
        logg.append([t, best])
        t += 1
    
    return logg

In [36]:
def show(his):
    input = np.array(his, dtype=object)
    x = input[:, 0]
    res = np.array(input[:, 1])
    res_tsp = []
    res_kp = []
    for indivs in res:
        res_tsp.append(Tasks[0].Cost(indivs[0]))
        res_kp.append(Tasks[1].Cost(indivs[1]))
    res_tsp = np.array(res_tsp)
    res_kp = np.array(res_kp)

    plt.figure(figsize=(20, 7))

    plt.subplot(1, 2, 1)
    plt.plot(x, res_tsp)
    plt.title('task 1: best = '+ str(res_tsp[-1]))

    plt.subplot(1, 2, 2)
    plt.plot(x, res_kp)
    plt.title('task 2: best = '+ str(res_kp[-1]))

    plt.show()

In [39]:
history = MFEA(30, 15, 0.3)
show(history)

AttributeError: 'numpy.ndarray' object has no attribute 'append'